<a href="https://colab.research.google.com/github/byiringiroscar/static_file_chatbot/blob/main/RssbChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [7]:
#check trans
# !pip install googletrans

In [8]:
# from googletrans import Translator
# translate_word = Translator()
# out = translate_word.translate('안녕하세요.', dest="en")
# out

In [9]:
#Fetch the content from the front page
content = requests.get('https://www.rssb.rw/').content

In [ ]:
#Preview of the Content
content

In [11]:
#Initiating the BS4 parser
page = BeautifulSoup(content, 'html.parser')

In [12]:
#Get all the links in the website
links = page.find_all('a')

In [13]:
len(links)

35

In [14]:
# main links fetching and main titles
list_of_links = [] #list()
list_of_titles = []

for item in links: #loop over the list of links
  if not str(item['href']) == '/':
    if len(item.text) > 0:
      if bool(re.search('rssb', str(item['href']))):
        if not item['href'].endswith('docx') and not item['href'].endswith('pdf') and not item['href'].endswith('xl') and not item['href'].endswith('xlsx') and not item['href'].endswith('html') and not item['href'].endswith('gov') and not item['href'].endswith('doc') and not item['href'].endswith('zip') and not item['href'].endswith('mdb') and not item['href'].endswith('xls'):
          if str(item['href']).startswith('http'):
            lin = str(item['href'])
            list_of_links.append(lin)
            list_of_titles.append(item.text)
            
          else:
            lin = str(item['href'])
            domain = 'https://www.rssb.rw' + lin
            list_of_links.append(domain)
            list_of_titles.append(item.text)
    
    

  
  

In [15]:
list_of_links

['https://www.imisanzu.rssb.rw/',
 'https://online.rssb.rw/index1152.php?menu=employer',
 'https://www.rssb.rw/rssb-invests-in-katapult-africa-seed-fund-nine-start-ups-selected-for-accelerator-program',
 'https://www.rssb.rw/rssb-remembers-former-csr-employees-killed-during-the-1994-genocide-against-the-tutsi',
 'https://online.rssb.rw/index1152.php?menu=employer',
 'https://www.imisanzu.rssb.rw/']

In [16]:

len(list_of_titles)

6

In [17]:
# take every link then go inside then get all links associated with it
def get_individual_link(fina_l): 
  try:
    if not fina_l.endswith('docx') and not fina_l.endswith('pdf') and not fina_l.endswith('xl') and not fina_l.endswith('xlsx') and not fina_l.endswith('html') and not fina_l.endswith('gov') and not fina_l.endswith('doc') and not fina_l.endswith('zip') and not fina_l.endswith('mdb') and not fina_l.endswith('xls'):
      content_individual_link = requests.get(fina_l).content # get content for each link
      page_individual_link = BeautifulSoup(content_individual_link, 'html.parser', from_encoding="iso-8859-1")
      final_link_individual = page_individual_link.findAll('a')
      for links_ind in final_link_individual:
        if not str(links_ind['href']) == '/':
          if len(links_ind.text) > 0:
            if str(links_ind['href']).startswith('http'):
              lin = str(links_ind['href'])
              list_of_links.append(lin)
              list_of_titles.append(links_ind.text)
            else:
              lin = str(links_ind['href'])
              domain = 'https://www.rssb.rw' + lin
              list_of_links.append(domain)
              list_of_titles.append(links_ind.text)
  except:
    pass



In [18]:
len(list_of_titles)

6

In [19]:
for i in range(len(list_of_links)):
  fina_links = list_of_links[i]
  fin_li_official = get_individual_link(fina_links)

In [20]:
len(list_of_links)

85

In [21]:
import re
real_link = []
for lin in list_of_links:
  if bool(re.search('rssb', lin)):
    if not lin.endswith('docx') and not lin.endswith('pdf') and not lin.endswith('xl') and not lin.endswith('xlsx') and not lin.endswith('html') and not lin.endswith('gov') and not lin.endswith('doc') and not lin.endswith('zip') and not lin.endswith('mdb') and not lin.endswith('xls'):
      real_link.append(lin)





In [22]:
len(real_link)

78

In [23]:
# remove duplicate in order to see all rssb what content inside then append it in list called list_of_links
real_link = list(set(real_link))
len(real_link)

27

In [24]:
def get_content_link_rssb_links(link_rssb):
  try:
    content_rssb = requests.get(link_rssb).content
    page_rssb = BeautifulSoup(content_rssb, 'html.parser')
    links_rssb = page_rssb.find_all('p')
    links_rssb_h_tag = page_rssb.find_all('h2')
    if len(links_rssb) > 0:
      for links_rssb in links_rssb:
        list_of_titles.append(links_rssb.text[50])
        list_of_links.append(link_rssb)
    elif links_rssb_h_tag:
      if len(links_rssb_h_tag) > 0:
        for links_rssb_h_tag in links_rssb_h_tag:
          list_of_titles.append(links_rssb_h_tag.text)
          list_of_links.append(link_rssb)
  except:
    pass

In [25]:
for lin_rssb in real_link:
  try:
    get_content_link_rssb_links(lin_rssb)
  except:
    pass


In [26]:

len(list_of_titles)
len(list_of_links)

101

In [27]:
df = pd.DataFrame()
df['title'] = list_of_titles
df['link'] = list_of_links
df['description'] = None

In [28]:
df.describe()

,title,link,description
count,101,101,0
unique,55,31,0
top,,https://www.rssb.rw#,NaN
freq,10,20,NaN


In [29]:
#Drop all the row with empty title columns
df = df.dropna(subset=['title'])

In [30]:
df.describe()

,title,link,description
count,101,101,0
unique,55,31,0
top,,https://www.rssb.rw#,NaN
freq,10,20,NaN


In [31]:
df.drop_duplicates(inplace=True, subset='title')

In [32]:
df.describe()

,title,link,description
count,55,55,0
unique,55,31,0
top,Check Contribution,https://www.rssb.rw/rssb-remembers-former-csr-...,NaN
freq,1,4,NaN


In [33]:
#Check whehter the link is full formated link
is_full_link = df.link.str.startswith('http')

#Check whether the link is pdf documents
is_pdf = df.link.str.contains('.pdf')

In [34]:
df['is_full_link'] = is_full_link #Create a new col
df['is_pdf'] = is_pdf #create a new col

In [35]:
df

,title,link,description,is_full_link,is_pdf
0,Check Contribution,https://www.imisanzu.rssb.rw/,None,True,False
1,Apply for Employee Number,https://online.rssb.rw/index1152.php?menu=empl...,None,True,False
2,a month agoRSSB invests in Katapult Africa See...,https://www.rssb.rw/rssb-invests-in-katapult-a...,None,True,False
3,7 months agoRSSB remembers former CSR employee...,https://www.rssb.rw/rssb-remembers-former-csr-...,None,True,False
4,Apply for employee number,https://online.rssb.rw/index1152.php?menu=empl...,None,True,False
5,Check for your contributions,https://www.imisanzu.rssb.rw/,None,True,False
6,Home,https://online.rssb.rw/,None,True,False
7,Employer Online Service,https://www.rssb.rw?menu=employer,None,True,False
8,Employee Online Service,https://www.rssb.rw?menu=employee,None,True,False
9,Voluntary Scheme Registration,https://www.rssb.rw?menu=voluntary,None,True,False


In [36]:
# data cleaning in dataframe
import re 

def preprocessing(titles : list):
  cleaned_titles = []
  for title in titles:
    title = title.strip()
    title = title.lower()
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    title = url_pattern.sub(r'', title)
    title = html_pattern.sub(r'', title)
    title = re.sub(r"[^\w\d'\s]+", ' ', title)
    cleaned_titles.append(title)
  return cleaned_titles


df['title'] = preprocessing(df['title'])

# Start Building the chatbot

## Building the intent file

In [37]:
question = """Guide me to {}?
Where do you get {}?
What is the link to the {}?
How do you get {}?
Who is in Charge of {}?
how can I get {}?
which way to get {}?
I need to get {}?
direct me to {}
please how can I {}?
I need to know more related with {}?
how can someone {}?
I need direction to {}?
{}"""

answers = """Here is the link {}.
You can use this link {} for more information.
thanks for contacting use this {} for more detail.
ok try this {} for more information.
take a look on this {}.
check out on this link {} for more detail."""

In [38]:
answers.split('\n')

['Here is the link {}.',
 'You can use this link {} for more information.',
 'thanks for contacting use this {} for more detail.',
 'ok try this {} for more information.',
 'take a look on this {}.',
 'check out on this link {} for more detail.']

In [39]:
# code for directory permanent
# import shutil

# shutil.rmtree('/content/pytorch-chatbot')

In [40]:
!git clone https://github.com/patrickloeber/pytorch-chatbot.git

fatal: destination path 'pytorch-chatbot' already exists and is not an empty directory.


In [41]:
import json
with open('/content/pytorch-chatbot/intents.json', 'r') as intent_file:
  intents__ = json.load(intent_file)

In [42]:
greet_intents = []
for intet in intents__['intents']:
  greet_intents.append(intet)


In [43]:
intents = []

for index in range(len(df)):
  intent = {}
  tag = df.iloc[index]['title']
  intent['tag'] = tag
  intent['patterns'] = list()
  intent['responses'] = list()
  # intent['responses'] = [f'You can use this link {df.iloc[index].link} for more information']
  for i in range(len(question.split('\n'))):
    intent['patterns'].append(question.split('\n')[i].format(intent['tag']))
  for i in range(len(answers.split('\n'))):
    intent['responses'].append(answers.split('\n')[i].format(df.iloc[index].link))

  intents.append(intent)

In [44]:
intents.extend(greet_intents)

In [45]:
file = {'intents': intents}
file

{'intents': [{'tag': 'check contribution',
   'patterns': ['Guide me to check contribution?',
    'Where do you get check contribution?',
    'What is the link to the check contribution?',
    'How do you get check contribution?',
    'Who is in Charge of check contribution?',
    'how can I get check contribution?',
    'which way to get check contribution?',
    'I need to get check contribution?',
    'direct me to check contribution',
    'please how can I check contribution?',
    'I need to know more related with check contribution?',
    'how can someone check contribution?',
    'I need direction to check contribution?',
    'check contribution'],
   'responses': ['Here is the link https://www.imisanzu.rssb.rw/.',
    'You can use this link https://www.imisanzu.rssb.rw/ for more information.',
    'thanks for contacting use this https://www.imisanzu.rssb.rw/ for more detail.',
    'ok try this https://www.imisanzu.rssb.rw/ for more information.',
    'take a look on this https:

In [46]:
import json

with open('/content/pytorch-chatbot/intents.json', 'w+') as intent_file:
  intent_file.seek(0)
  intent_file.truncate()
  json.dump(file, intent_file)

In [47]:
%cd pytorch-chatbot/

/content/pytorch-chatbot


In [48]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
!python train.py

1566 patterns
59 tags: ['', '7 months agorssb remembers former csr employees killed during the 1  rwanda social security board  rssb  has remembered its former employees who were killed during the 1994 genocide against the tutsi  the event was orga  read more  ', 'a month agorssb invests in katapult africa seed fund  nine start u  on friday 4th november 2022  katapult africa officially launched its accelerator program in kigali  rwanda together with its partners  norad   norwegi  read more  ', 'abanyarwanda barakangurirwa kwihutira kwishyura mituweli ya 2020 2021 mu gihe hasigaye  iminsi  24 ngo hatangire umwaka wa mutuelle de santã  wa 2020 2021   imiryango ibihumbi 42 777 niyo yonyine imaze kwishyura umusanzu wâ ubwisungane mu kwivuza  mutuelle de santã    ya 2  7 months ago', 'abanyarwanda barakangurirwa kwihutira kwishyura mituweli ya 2020 2021 mu gihe hasigaye  iminsi  24 ngo hatangire umwaka wa mutuelle de santã  wa 2020 2021   imiryango ibihumbi 42 777 niyo yonyine imaze kwishyu

In [50]:
import random
import json

import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Oscar"
print("Let's chat! (type 'quit' to exit)")

def query(question : str):
  # sentence = "do you use credit cards?"
  sentence = question

  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device)

  output = model(X)
  _, predicted = torch.max(output, dim=1)

  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]
  if prob.item() > 0.75:
      for intent in intents['intents']:
          if tag == intent["tag"]:
              return f"{bot_name}: {random.choice(intent['responses'])}"
  else:
      return f"{bot_name}: I do not understand..."

Let's chat! (type 'quit' to exit)


In [51]:
response = query('how are you ?')

In [52]:
response

'Oscar: Hey :-)'

In [53]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:

import requests
import json


class translator:
    api_url = "https://translate.googleapis.com/translate_a/single"
    client = "?client=gtx&dt=t"
    dt = "&dt=t"

    #fROM English to Kinyarwanda
    def translate(text : str , target_lang : str, source_lang : str):
        sl = f"&sl={source_lang}"
        tl = f"&tl={target_lang}"
        r = requests.get(translator.api_url+ translator.client + translator.dt + sl + tl + "&q=" + text)
        return json.loads(r.text)[0][0][0]

In [55]:
resp = translator.translate(text='Mpereza amakuru yerekeye Belise', target_lang='fr', source_lang='rw')

In [56]:
resp

'Je recherche des informations sur Belize'

In [57]:
from langdetect import detect

def process_question(text : str):
  if text.lower() == 'hi' or text.lower() == 'hey' or text.lower() == 'how are you' or text.lower() == 'hello':
    resp = text.lower()
    source_lang = 'en'
    return resp, source_lang
  else:
    source_lang = detect(text)
    resp = translator.translate(text=text, target_lang='en', source_lang=source_lang)
    return resp, source_lang

def process_answer(text : str, source_lang):
  resp = translator.translate(text=text, target_lang=source_lang, source_lang='en')
  return resp

In [58]:
def process(QUESTION: str):
  USER_QUERY, SL = process_question(QUESTION) #Translate the original question into english and store the source lang
  RESPONSE = query(USER_QUERY) #Asking th chatbot question
  ORIGINAL_RESPONSE = process_answer(RESPONSE, SL)
  return ORIGINAL_RESPONSE
a = 'SENIOR MANAGEMENT'
process(a.lower())

'Oscar: kijk eens op https://www.rssb.rw/management.'

#FAST API

In [59]:
!pip install fastapi pyngrok uvicorn nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
!ngrok authtoken 2HJTD4eU4nX4kqE0VD1YVDlysnq_4pvNxMFU5pe3Q4ERkSeHP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [61]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from IPython.core.interactiveshell import asyncio
import uvicorn
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
import schemas
app = FastAPI()
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

templates = Jinja2Templates(directory='/content/templates')
app.mount("/content/static", StaticFiles(directory="/content/static"), name="static")


# storage = FastAPI(title='FASTAPI')
# fakeDatabase = {
#     1: {'task': "hello"},
# }

@app.get('/', response_class=HTMLResponse)
def home(request: Request):
    return templates.TemplateResponse("base.html", {"request": request})
@app.post("/predict")
async def predict(item: schemas.Item):
    response = process(item.message)
    message = {"answer": response}
    return message



# @storage.get('/')
# async def index():
#   return {"data": "successfully"}
# @storage.get("/{lang}")
# async def getItem(lang: str):
#   if lang:
#     sol = process(lang)
#     new_id = len(fakeDatabase.keys()) + 1
#     fakeDatabase[new_id] = {'task': sol}
#     return fakeDatabase
#   else:
#     return {"data": "failed"}

  

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print("REST API started")
print("Your public API URL:", ngrok_tunnel.public_url)
print("You can for example open the following URL in your browser: {}?target_lang=en&text=Hallo%20Welt".format(ngrok_tunnel.public_url))

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [1048]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


REST API started
Your public API URL: http://61d0-34-80-25-154.ngrok.io
You can for example open the following URL in your browser: http://61d0-34-80-25-154.ngrok.io?target_lang=en&text=Hallo%20Welt
